# Evaluate age & gender classifier with test data

### Source
* agegender_train.ipynb
* agegender_train.py
* agegender_predict.py

### Setup

Remember: 

* select GPU used for model training and
* run jupyter notebook on the port that is not used e.g.

$    jupyter notebook --port 5555

In [ ]:
# select GPU used for model training
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" # first gpu
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
print(os.environ["CUDA_VISIBLE_DEVICES"])

In [ ]:
# import cv2
import sys
import numpy as np
import os

os.environ['KERAS_BACKEND'] = 'tensorflow'

# from keras.applications.vgg16 import VGG16
# from keras.preprocessing import image
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt

In [ ]:
# from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

## Mode

In [ ]:
ANNOTATIONS=''
DATASET_NAME=''
MODELS=""
DATASET_ROOT_PATH=""
OPTIONAL_MODE=""
DATA_AUGUMENTATION=False

## Argument

In [ ]:
# >> FORMAT: sys_argv = [ANNOTATIONS, MODELS, DATASET_NAME, DATASET_ROOT_PATH, EXTRA_MODE]
# "usage: python agegender_predict.py [gender/age/age101/emotion] [inceptionv3/vgg16/squeezenet/octavio] [adience/imdb/utk/appareal/vggface2/empty] [datasetroot(optional)] [benchmark/caffemodel(optional)]"
from utils import get_local_dataset_root_path
# sys_argv = ["gender", "vgg16", "imdb", get_local_dataset_root_path()]
# sys_argv = ["gender", "vgg16", "utk", get_local_dataset_root_path()]
# sys_argv = ["gender", "inceptionv3", "imdb", get_local_dataset_root_path()]
sys_argv = ["gender", "squeezenet", "imdb", get_local_dataset_root_path()]

In [ ]:
start_index = 0
if len(sys_argv) >= start_index+3:
  ANNOTATIONS = sys_argv[start_index]
  MODELS = sys_argv[start_index+1]
  DATASET_NAME = sys_argv[start_index+2]
  if len(sys_argv) >= start_index+4:
    DATASET_ROOT_PATH=sys_argv[start_index+3]
  if len(sys_argv) >= start_index+5:
    OPTIONAL_MODE=sys_argv[start_index+4]
else:
  print("usage: python agegender_predict.py [gender/age/age101/emotion] [inceptionv3/vgg16/squeezenet/octavio] [adience/imdb/utk/appareal/vggface2/empty] [datasetroot(optional)] [benchmark/caffemodel(optional)]")
  sys.exit(1)

if ANNOTATIONS!="gender" and ANNOTATIONS!="age" and ANNOTATIONS!="age101" and ANNOTATIONS!="emotion":
  print("unknown annotation mode");
  sys.exit(1)

if MODELS!="inceptionv3" and MODELS!="vgg16" and MODELS!="squeezenet" and MODELS!="mobilenet" and MODELS!="octavio":
  print("unknown network mode");
  sys.exit(1)

if DATASET_NAME!="adience" and DATASET_NAME!="imdb" and DATASET_NAME!="utk" and DATASET_NAME!="appareal" and DATASET_NAME!="vggface2" and DATASET_NAME!="empty":
  print("unknown dataset name");
  sys.exit(1)

if OPTIONAL_MODE!="" and OPTIONAL_MODE!="benchmark" and OPTIONAL_MODE!="caffemodel":
  print("unknown optional mode");
  sys.exit(1)

if DATASET_NAME=="empty":
    DATASET_NAME=""
else:
    DATASET_NAME='_'+DATASET_NAME

In [ ]:
CUSTOM_MODEL_HDF5 = ''
CUSTOM_MODEL_HDF5 = '/home/krittametht/__backup_YoloKerasFaceDetection_trained/_imdb_cleaned_2_squeeznet_sgd_lr_1e-3/agegender_gender_squeezenet_imdb.hdf5'

In [ ]:
print(ANNOTATIONS, MODELS, DATASET_NAME, DATASET_ROOT_PATH, OPTIONAL_MODE)
if len(CUSTOM_MODEL_HDF5) > 0:
    print(CUSTOM_MODEL_HDF5)

## Converting

In [ ]:
AUGUMENT=""
if(DATA_AUGUMENTATION):
  AUGUMENT="augumented"

MODEL_HDF5=DATASET_ROOT_PATH+'pretrain/agegender_'+ANNOTATIONS+'_'+MODELS+DATASET_NAME+AUGUMENT+'.hdf5'
ANNOTATION_WORDS='words/agegender_'+ANNOTATIONS+'_words.txt'

if(ANNOTATIONS=="emotion"):
    ANNOTATION_WORDS='words/emotion_words.txt'

if(MODELS=="octavio"):
    if(ANNOTATIONS=="emotion"):
        MODEL_HDF5=DATASET_ROOT_PATH+'pretrain/fer2013_mini_XCEPTION.102-0.66.hdf5'
    if(ANNOTATIONS=="gender"):
        MODEL_HDF5=DATASET_ROOT_PATH+'pretrain/gender_mini_XCEPTION.21-0.95.hdf5'

# --added
if len(CUSTOM_MODEL_HDF5) > 0:
    MODEL_HDF5 = CUSTOM_MODEL_HDF5
        
if(MODELS=="mobilenet"):
    import keras
    from keras.utils.generic_utils import CustomObjectScope
    with CustomObjectScope({'relu6': keras.applications.mobilenet.relu6,'DepthwiseConv2D': keras.applications.mobilenet.DepthwiseConv2D}):
        keras_model = load_model(MODEL_HDF5)
else:
    keras_model = load_model(MODEL_HDF5)
keras_model.summary()

## Convert to caffe model

In [ ]:
if OPTIONAL_MODE=="caffemodel":
    os.environ["GLOG_minloglevel"] = "2"
    import caffe
    import keras2caffe
    prototxt=DATASET_ROOT_PATH+'pretrain/agegender_'+ANNOTATIONS+'_'+MODELS+DATASET_NAME+'.prototxt'
    caffemodel=DATASET_ROOT_PATH+'pretrain/agegender_'+ANNOTATIONS+'_'+MODELS+DATASET_NAME+'.caffemodel'
    keras2caffe.convert(keras_model, prototxt, caffemodel)

## Benchmark

In [ ]:
if OPTIONAL_MODE=="benchmark":
    BENCHMARK_DATASET_NAME="imdb"
    BENCHMARK_DATASET_TARGET="validation"
    BATCH_SIZE=64

    shape = keras_model.layers[0].get_output_at(0).get_shape().as_list()

    disp_generator = ImageDataGenerator(rescale=1.0 / 255).flow_from_directory(
       DATASET_ROOT_PATH+'dataset/agegender_'+BENCHMARK_DATASET_NAME+'/annotations/'+ANNOTATIONS+'/'+BENCHMARK_DATASET_TARGET,
       target_size=(shape[1], shape[2]),
       batch_size=BATCH_SIZE,
       class_mode='categorical'
    )

    DISTRIBUTION_FILE=DATASET_ROOT_PATH+'pretrain/benchmark_'+ANNOTATIONS+"_"+MODELS+DATASET_NAME+'.png'

    fig = plt.figure()
    ax1 = fig.add_axes((0.1, 0.6, 0.8, 0.3))
    ax2 = fig.add_axes((0.1, 0.1, 0.8, 0.3))
    ax1.tick_params(labelbottom="on")
    ax2.tick_params(labelleft="on")

    max_cnt=len(disp_generator.filenames)
    #max_cnt=10

    x=np.zeros((max_cnt))
    y=np.zeros((max_cnt))
    t=np.zeros((max_cnt))

    cnt=0
    heatmap=np.zeros((len(disp_generator.class_indices),len(disp_generator.class_indices)))
    for x_batch, y_batch in disp_generator:
        for i in range(BATCH_SIZE):
            x[cnt]=y_batch[i][0]
            t[cnt]=y_batch[i].argmax()

            data=x_batch[i]
            data.shape = (1,) + data.shape
            pred = keras_model.predict(data)[0]
            cls = pred.argmax()

            y[cnt]=cls

            heatmap[int(y[cnt]),int(t[cnt])]=heatmap[int(y[cnt]),int(t[cnt])]+1

            cnt=cnt+1
            print(""+str(cnt)+"/"+str(max_cnt)+" ground truth:"+str(y_batch[i].argmax())+" predicted:"+str(cls))
            if cnt>=max_cnt:
                break
        if cnt>=max_cnt:
            break

    ax1.pcolor(heatmap, cmap=plt.cm.Blues)
    if heatmap.shape[0]<=2:
        for y in range(heatmap.shape[0]):
            for x in range(heatmap.shape[1]):
                ax1.text(x + 0.5, y + 0.5, '%.4f' % heatmap[y, x],
                    horizontalalignment='center',
                    verticalalignment='center',
                )

    ax1.set_title('ground truth vs predicted '+ANNOTATIONS)
    ax1.set_xlabel(ANNOTATIONS+'(ground truth)')
    ax1.set_ylabel(ANNOTATIONS+'(predicted)')
    ax1.legend(loc='upper right')

    ax2.hist(t, bins=len(disp_generator.class_indices))
    ax2.set_title('distribution of ground truth '+ANNOTATIONS)
    ax2.set_xlabel(ANNOTATIONS+'(ground truth)')
    ax2.set_ylabel('count')
    ax2.legend(loc='upper right')

    fig.savefig(DISTRIBUTION_FILE)
    sys.exit(1)

## Evaluate

In [ ]:
BATCH_SIZE = 16
# BATCH_SIZE = 32

# BENCHMARK_DATASET_NAME="imdb"
BENCHMARK_DATASET_NAME="utk"
# BENCHMARK_DATASET_TARGET="validation"
BENCHMARK_DATASET_TARGET="all"

shape = keras_model.layers[0].get_output_at(0).get_shape().as_list()
# print(shape)

benchmark_dataset_path = DATASET_ROOT_PATH+'dataset/agegender_'+BENCHMARK_DATASET_NAME+'/annotations/'+ANNOTATIONS+'/'+BENCHMARK_DATASET_TARGET

# >> custom path
# benchmark_dataset_path = DATASET_ROOT_PATH+'dataset/agegender_'+BENCHMARK_DATASET_NAME+'/annotations/'+'gender_rm_age_001_to_010/all'
# benchmark_dataset_path = DATASET_ROOT_PATH+'dataset/agegender_'+BENCHMARK_DATASET_NAME+'/annotations/'+'gender/all'
# benchmark_dataset_path = DATASET_ROOT_PATH+'dataset/agegender_'+BENCHMARK_DATASET_NAME+'/annotations/'+'gender/validation'
# benchmark_dataset_path = DATASET_ROOT_PATH+'dataset/agegender_'+BENCHMARK_DATASET_NAME+'_crop_m-4'+'/annotations/'+'gender/all'
# benchmark_dataset_path = DATASET_ROOT_PATH+'dataset/agegender_'+BENCHMARK_DATASET_NAME+'_crop_m-4'+'/annotations/'+'gender_rm_age_001_to_005/all'
# benchmark_dataset_path = DATASET_ROOT_PATH+'dataset/agegender_'+BENCHMARK_DATASET_NAME+'_crop_m-4'+'/annotations/'+'gender_rm_age_001_to_010/all'
benchmark_dataset_path = DATASET_ROOT_PATH+'dataset/KBTG_staff_photo_resize/defined_crop_m-4/all'
print(benchmark_dataset_path)

In [ ]:
test_datagen = ImageDataGenerator(
    rescale=1.0 / 255
  )

test_generator = test_datagen.flow_from_directory(
    benchmark_dataset_path,
    target_size=(shape[1], shape[2]),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)
# print(test_generator.target_size)

test_data_n = len(test_generator.filenames)

print("Test data count : "+str(test_data_n))

## Evaluate

In [ ]:
eval_results = keras_model.evaluate_generator(
    generator=test_generator,
    verbose=1,
    steps=test_data_n//BATCH_SIZE
  )

In [ ]:
print(test_generator.directory)
for a,b in zip(keras_model.metrics_names, eval_results):
    print(a, ":\t", "%.6f" % b)